# Explore Ticker Data Compilation. 

---

## Project goal.

The objective is to __explore automating the process of data collecting, preprocessing, and dashboard update for market analysis__ to save time for more important task. The preprocessing functions can be applied across data collected from any platform as long the __date__ and __closing price__ variables exist. 

The functions created here are then integrated with the Luigi pipeline. You can refer to the python file [luigi pipeline.py][luigi] on [Github][github].

I might use specific data to train and test the machine learning (ML) model. It is obvious that ML cannot accurately estimate the market price change due to the complexity of the problem. However, the intention is to practise applying the ML concepts and gain experience in building a ML system and application which include data streaming, processing, performance measure, model optimisation, and model maintainance and update. 

The ML model will also be used along with NLP model to predict news sentiment, and summarise news and reports. 

__Additional info about the data:__

- The __closing price__ data that I collected is adjusted based on the dividend. 
- I will only __update most of the data on yearly basis__ since the data will be averaged across a range of years, except for certain data which requires monthly update for ML project. Since there are a lot of data to collect, I created this project, though I don't often update most of the data. 

## Table of content. 

1. [Configuration, overall.](#Configuration,-overall.)
2. [Collect data by navigating the website with Selenium.](#Collect-data-by-navigating-the-website-with-Selenium.)
3. Preprocess the price change data:
    * [Configuration for price data.](#Configuration-for-price-data.)
    * [Functions for price change data preprocessing.](#Functions-for-price-change-data-preprocessing.)
    * [Read and preprocess the data.](#Read-and-preprocess-the-data.)
    * [Create pivot tables for price change.](#Create-pivot-tables-for-price-change.)
    * [Compute statistical summary on the pivot data, price change.](#Compute-statistical-summary-on-the-pivot-data,-price-change.)
4. Preprocess the volume data:
    * [Configuration for volume data.](#Configuration-for-volume-data.)
    * [Functions for volume data preprocessing.](#Functions-for-volume-data-preprocessing.)
    * [Create pivot tables for volume.](#Create-pivot-tables-for-volume.)
    * [Compute statistical summary on the pivot data, volume.](#Compute-statistical-summary-on-the-pivot-data,-volume.)
5. Preprocess the holidays, observances, and special periods data. 
    * [Configuration for holidays, observances, and special periods data.](#Configuration-for-holidays,-observances,-and-special-periods-data.)
    * [Function for tracing the holidays, observances, and special periods.](#Function-for-tracing-the-holidays,-observances,-and-special-periods.)
    * [Get the holiday, observance, and special period dates.](#Get-the-holiday,-observance,-and-special-period-dates.)
    * [Trace the FirstTrdrDoM and Santa Rally dates.](#Trace-the-FirstTrdrDoM-and-Santa-Rally-dates.)
    * [Trace the holiday and observance dates.](#Trace-the-holiday-and-observance-dates.)
    * [Create pivot tables for holidays, observances, and special periods.](#Create-pivot-tables-for-holidays,-observances,-and-special-periods.)
    * [Compute statistical summary on the pivot data; holidays, observances, and special periods.](#Compute-statistical-summary-on-the-pivot-data;-holidays,-observances,-and-special-periods.)
6. Save the data into Excel file on multiple sheets. 
    * [Configuration for writing to Excel file.](#Configuration-for-writing-to-Excel-file.)
    * [Function for saving multiple data into Excel for verification.](#Function-for-saving-multiple-data-into-Excel-for-verification.)
    * [Save to Excel.](#Save-to-Excel.)
7. [Simple file verification.](#Simple-file-verification.)
8. [Notes for future reference.](#Notes-for-future-reference.)

[luigi]:https://github.com/lionelcub/compile_ticker_data/blob/master/luigi_pipeline.py
[github]:https://github.com/lionelcub/compile_ticker_data

In [1]:
# For datetime preprocessing. 
from datetime import datetime, timedelta

# For data preprocessing. 
import numpy as np
import pandas as pd

# For identifying holidays date. 
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, USLaborDay, \
    USColumbusDay, USThanksgivingDay

# For file management. 
import os
import shutil

# Pandas DF preview configuration. 
pd.set_option("display.max_columns", 100, "display.max_rows", 100)

---

## Configuration, overall.

Return to [Table of content](#Table-of-content.)

Create a section for configuration. This section will be saved in a file called __config.py__ under __configuration__ folder to organise my task and file later. This is just a high level configuration. There are other configuration sections reserved for a much detailed configuration. 

In [2]:
# Specify the year range to collect the data. 
# Be careful when assigning the 'end_yr' to 'range()'. You need to add 1 to it. 
start_yr, end_yr = (1999,2019)
ls_yrStr = [str(yr) for yr in range(start_yr, end_yr + 1)]

yr_range = ['MaxYr', 'R20Yr', 'R15Yr', 'R10Yr', 'R5Yr']
start_yr_range = [start_yr, end_yr-19, end_yr-14, end_yr-9, end_yr-4] 


# Start and end trading timestamp. 
start_date = {
    'mo': int(datetime.timestamp(datetime(start_yr - 1, 11, 30))),
    'wk': int(datetime.timestamp(datetime(start_yr - 1, 12, 28))),
    'dy': int(datetime.timestamp(datetime(start_yr - 1, 12, 23)))}

end_date = {
    'mo': int(datetime.timestamp(datetime(end_yr + 1, 1, 1))),
    'wk': int(datetime.timestamp(datetime(end_yr + 1, 1, 4))),
    'dy': int(datetime.timestamp(datetime(end_yr + 1, 1, 4)))}


# Ticker info. 
ETF_folder = 'dataset/ETF_sector'
ticker = 'SPY'
ticker_freq = ['1mo', '1wk', '1d']
yahoo_version = 'v7'


# Path. 
project_path = os.getcwd()
driver_path = os.path.join(project_path, 'system/chromedriver')

---

## Collect data by navigating the website with Selenium. 

Return to [Table of content](#Table-of-content.)

I have __2 approaches__ to data collecting. One is to use `Selenium` to download the data by navigating the site. Another is to use `curl` commandline to download the data via a direct download link. The 2nd approach is easier and faster, but some websites may not provide the direct download link. 

The following code is a sample of the 1st approach. I found a site where I can download the data directly using `curl`, but in case I need it, I can work on top of these functions. However, the __chrome driver__ might have to update as well for the `Selenium` to access. 

I also automate the process of creating a new folder, and moving and renaming files. 

In [3]:
def download_ticker_data(driver_path, url):
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import Select
    from time import sleep
    
    # Open Chrome driver to access the webpage. 
    driver = webdriver.Chrome(executable_path=driver_path)
    driver.get(url)
    print('Data downloaded:\n', url, '\n')
    
    # Try the Download button and click it. 
    try: driver.find_element_by_css_selector("svg[data-icon='download']").click()
    except: sleep(5)
        
    driver.close()
    
    
def move_file(ETF_folder, ticker, ticker_filename):
    # Create new folder for the specific ticker. 
    try: os.makedirs(f'{ETF_folder}/{ticker}')
    except: pass

    # Go to home directory. 
    os.chdir('/Users/lioneltay')

    # Rename file. 
    os.rename(f'Downloads/{ticker}.csv', f'Downloads/{ticker_filename}')

    # Create directory path. 
    home_path = os.getcwd() 
    trdr_path = os.path.join(home_path, f'Google Drive/TR_research/research_journal/{ETF_folder}/{ticker}') 
    downloads_path = os.path.join(home_path, f'Downloads/{ticker_filename}')

    # Move file from the 'Downloads' directory to the destination. 
    try: shutil.move(downloads_path, trdr_path)
    except: os.replace(downloads_path, os.path.join(trdr_path, ticker_filename))

    # Revert back to project directory. 
    os.chdir(project_path)

    
def multiple_download(driver_path, ticker, ticker_freq, start_date, end_date):
    for i, freq in enumerate(start_date.keys()):
        # Webpage link to download the ticker data. 
        # ticker_yahoo = f"https://finance.yahoo.com/quote/{ticker}/history"
        # ticker_param = f"?period1={start_date[freq]}&period2={end_date[freq]}&interval={ticker_freq[i]}&filter=history&frequency={ticker_freq[i]}"
        # ticker_url = "".join([ticker_yahoo, ticker_param])
        
        # Direct download link if Selenium fails to detect the HTML line. 
        ticker_download = f'https://query1.finance.yahoo.com/{yahoo_version}/finance/download/{ticker}?'
        ticker_download_param = f'period1={start_date[freq]}&period2={end_date[freq]}&interval={ticker_freq[i]}&events=history'
        ticker_download_url = "".join([ticker_download, ticker_download_param])
        
        # Download data and rename file. 
        download_ticker_data(driver_path, ticker_download_url) 
        ticker_filename = f'{ticker}_{ticker_freq[i]}.csv'
        
        # Move the file to project directory. 
        move_file(ETF_folder, ticker, ticker_filename)
        print('File moved\n')
        

multiple_download(driver_path, ticker, ticker_freq, start_date, end_date)

---

## Preprocess the price change data. 

### Configuration for price data.

Return to [Table of content](#Table-of-content.)

This __configuration section focuses on closing price change__. I will create __4 types__ of datasets, __monthly, weekly, daily (by trading day), and daily (by weekday)__. Different dataset will be used for different preprocessing procedure and purpose of analysis. 

This section will be saved in __config.py__ under __configuration__ folder to organise my task and file later.

In [4]:
# Store keys for dictionary indexing. 
freq_keys = ['monthly', 'weekly', 'daily_byTrdrDay', 'daily_byWeekday']

# Column names for creating pivot tables. 
freq_cols = ['month', 'week', 'trdr_day', 'weekday']

# To store ticker data.
df_ticker = {
    freq_keys[0]: pd.read_csv(f'{ETF_folder}/{ticker}/{ticker}_1mo.csv', parse_dates=['Date']),
    freq_keys[1]: pd.read_csv(f'{ETF_folder}/{ticker}/{ticker}_1wk.csv', parse_dates=['Date']),
    freq_keys[2]: pd.read_csv(f'{ETF_folder}/{ticker}/{ticker}_1d.csv', parse_dates=['Date']),
    freq_keys[3]: pd.read_csv(f'{ETF_folder}/{ticker}/{ticker}_1d.csv', parse_dates=['Date'])
}

# To store pivot tables.
pivot_ticker = {
    freq_keys[0]: None, 
    freq_keys[1]: None, 
    freq_keys[2]: None, 
    freq_keys[3]: None
}

# To store statistical summary of pivot tables. 
pivot_stats = {
    freq_keys[0]: None, 
    freq_keys[1]: None, 
    freq_keys[2]: None, 
    freq_keys[3]: None
}

---

### Functions for price change data preprocessing. 

Return to [Table of content](#Table-of-content.)

Below is the created functions __for preprocessing and creating a statistical summary__. Each function has its documentation. I __reduce the running time to O(N)__ as much as possible for python code except for pandas functions. The functions will be saved in __preprocessing.py__ under __autoprocess_ticker__ folder. 

In [5]:
def init_preprocess(df_dict, freq_keys):
    '''
    Purpose: 
        1. Cast the column names to lowercase.
        2. Compute the price change.
        3. Extract year from the date column. 
        4. Create additional columns: month, week, trdr_day, weekday for 
           monthly, weekly, and daily data.
    
    Input  :
        df_dict      : Dictionary. Should contain ticker dataframe.
        freq_keys    : List of dictionary keys. Must contain monthly/weekly/daily. 
        
    Return :
        None.
    '''
    
    for freq in freq_keys:
        # Cast column names to lowercase. 
        df_dict[freq].columns = list(map(str.lower, df_dict[freq].columns))

        # Compute price difference.
        df_dict[freq]['price_diff'] = df_dict[freq]['adj close'].pct_change(periods=1)

        # Extract year from date. 
        df_dict[freq]['year'] = df_dict[freq]['date'].dt.year

    # Monthly data. 
    df_dict[freq_keys[0]][freq_cols[0]] = df_dict[freq_keys[0]]['date'].dt.month
    
    # Weekly data. 
    df_dict[freq_keys[1]][freq_cols[1]] = df_dict[freq_keys[1]]['date'].dt.week
    
    # Daily data (byMonth).
    df_dict[freq_keys[2]][freq_cols[0]] = df_dict[freq_keys[2]]['date'].dt.month
    df_dict[freq_keys[2]][freq_cols[1]] = df_dict[freq_keys[2]]['date'].dt.week
    df_dict[freq_keys[2]][freq_cols[2]] = df_dict[freq_keys[2]].set_index(keys='date')\
                                          .groupby(by=pd.Grouper(freq='M')).cumcount().values
    
    # Daily data (byWeekday).
    df_dict[freq_keys[3]][freq_cols[1]] = df_dict[freq_keys[3]]['date'].dt.week
    df_dict[freq_keys[3]][freq_cols[3]] = df_dict[freq_keys[3]]['date'].dt.weekday

    

def create_pivot(df_dict, pivot_dict, freq_keys, freq_cols, value):
    '''
    Purpose: 
        Create pivot table with 'year' as columns and 'month/week/trdr_day/weekday' 
        as index.
    
    Input  :
        df_dict        : Dictionary. Should contain ticker dataframe.
        pivot_dict     : Dictionary. To contain the pivot tables.
        freq_keys      : List of dictionary keys. Must contain monthly/weekly/daily. 
        freq_cols      : List. Must contain month / week / trdr_day / weekday.
        Value          : String. The column to perform computation on.
        
    Return :
        None.
        
    Note   :
        No aggregation should be performed although 'aggfunc' is assigned 'mean'. 
        The value should be exactly the same as the data from df_ticker. 
    '''
    
    column = 'year'
    
    for col, freq in zip(freq_cols, freq_keys):
        index = [col]
        if col == 'trdr_day': index = ['month', col]
        elif col == 'weekday': index = ['week', col]
    
        pivot_dict[freq] = df_dict[freq].pivot_table(values=value, index=index, 
                                                     columns=column, aggfunc='mean')


        
def summarise_pivot(pivot_dict, pivot_dict_stats, freq_keys, start_yr_range, end_yr):
    '''
    Purpose: 
        Perform statistical computation for month/week/trdr_day/weekday.
    
    Input  :
        pivot_dict      : Dictionary. Should contain pivot tables.
        pivot_dict_stats: Dictionary. To contain the summarised data from pivot_dict.
        freq_keys       : List of dictionary keys. Must contain monthly/weekly/daily. 
        start_yr_range  : List. Range of starting year to summarise the data on. 
        end_yr          : Int. Ending year to summarise the data on. 

    Return :
        None.
        
    Note   :
        There are cells that contain NaN. A warning will raise if the entire cells 
        for calculating the mean or standard deviation are NaN. So far, the resulted 
        calculation contains no error. The 'count()' function also outputs the correct 
        counts. 
    '''
    
    for freq in freq_keys:
        repeated_start_yr = False
        
        for i, start_yr in enumerate(start_yr_range):
            if start_yr < start_yr_range[0]: 
                continue
            
            stats_key = f'{freq}_{yr_range[i]}'
            if start_yr == start_yr_range[0]:
                stats_key = freq 
                if repeated_start_yr:
                    continue
                repeated_start_yr = True
        
            # Compute the average price change across years. 
            # Store it as a DataFrame object. 
            pivot_dict_stats[stats_key] = pd.DataFrame(pivot_dict[freq].loc[:,start_yr:end_yr].mean(axis=1), 
                                                       columns=['avg_diff'])

            # Compute median price change.
            pivot_dict_stats[stats_key]['med_diff'] = pivot_dict[freq].loc[:,start_yr:end_yr].median(axis=1)

            # Compute total price change.
            pivot_dict_stats[stats_key]['tot_diff'] = pivot_dict[freq].loc[:,start_yr:end_yr].sum(axis=1)

            # Compute max and min price change.
            pivot_dict_stats[stats_key]['max_diff'] = pivot_dict[freq].loc[:,start_yr:end_yr].max(axis=1)
            pivot_dict_stats[stats_key]['min_diff'] = pivot_dict[freq].loc[:,start_yr:end_yr].min(axis=1)

            # Compute standard deviation. 
            pivot_dict_stats[stats_key]['std_diff'] = pivot_dict[freq].loc[:,start_yr:end_yr].std(axis=1)

            # Indicate whether the average price change is positive or negative. 
            pivot_dict_stats[stats_key]['up_overall'] = 0
            pivot_dict_stats[stats_key].loc[pivot_dict_stats[stats_key]['avg_diff'] > 0,'up_overall'] = 1

            # Compute average positive price change.
            df = pivot_dict[freq][pivot_dict[freq] > 0]
            pivot_dict_stats[stats_key]['pos_avg_diff'] = df.loc[:,start_yr:end_yr].mean(axis=1)

            # Count positive price change. 
            up_counts = df.loc[:,start_yr:end_yr].count(axis=1)
            pivot_dict_stats[stats_key]['up_counts'] = up_counts

            # Compute average negative price change.
            df = pivot_dict[freq][pivot_dict[freq] < 0]
            pivot_dict_stats[stats_key]['neg_avg_diff'] = df.loc[:,start_yr:end_yr].mean(axis=1)

            # Count negative price change. 
            down_counts = df.loc[:,start_yr:end_yr].count(axis=1)
            pivot_dict_stats[stats_key]['down_counts'] = down_counts

            # Compute the probability of up and down. 
            prob = (up_counts / (up_counts + down_counts)).round(4)
            pivot_dict_stats[stats_key]['up_prob'] = prob
            pivot_dict_stats[stats_key]['down_prob'] = 1 - prob

---

### Read and preprocess the data. 

Return to [Table of content](#Table-of-content.)

In [6]:
init_preprocess(df_ticker, freq_keys)

In [7]:
# Preview.
df_ticker['monthly'].head()

,date,open,high,low,close,adj close,volume,price_diff,year,month
0,1998-12-01,116.12500,124.75000,113.750000,123.31250,83.100800,156026700,NaN,1998,12
1,1999-01-01,123.37500,128.50000,120.375000,127.65625,86.313889,141419000,0.038665,1999,1
2,1999-02-01,128.68750,128.84375,121.328125,123.56250,83.545898,164624900,-0.032069,1999,2
3,1999-03-01,123.65625,132.62500,121.781250,128.37500,86.799858,148191100,0.038948,1999,3
4,1999-04-01,129.68750,137.50000,128.125000,133.25000,90.314575,156755700,0.040492,1999,4


In [8]:
# Preview.
df_ticker['weekly'].head()

,date,open,high,low,close,adj close,volume,price_diff,year,week
0,1998-12-28,123.250000,124.750000,122.00000,123.31250,83.376900,21740600,NaN,1998,53
1,1999-01-04,123.375000,128.500000,121.71875,127.75000,86.377289,36948400,0.035986,1999,1
2,1999-01-11,127.687500,127.687500,120.37500,124.37500,84.095291,45376100,-0.026419,1999,2
3,1999-01-18,125.296875,127.937500,121.78125,122.56250,82.869736,27529300,-0.014573,1999,3
4,1999-01-25,123.281250,128.296875,121.90625,127.65625,86.313889,31565200,0.041561,1999,4


In [9]:
# Preview.
df_ticker['daily_byTrdrDay'].head()

,date,open,high,low,close,adj close,volume,price_diff,year,month,week,trdr_day
0,1998-12-22,120.40625,121.218750,119.1875,120.68750,81.601967,5461100,NaN,1998,12,52,0
1,1998-12-23,121.18750,123.218750,120.8125,123.21875,83.313499,7791000,0.020974,1998,12,52,1
2,1998-12-24,123.15625,123.250000,122.5000,122.68750,82.954323,1507100,-0.004311,1998,12,52,2
3,1998-12-28,123.25000,123.312500,122.0000,122.37500,82.742989,4203600,-0.002548,1998,12,53,3
4,1998-12-29,122.71875,124.484375,122.1250,124.31250,84.053032,3935800,0.015833,1998,12,53,4


In [10]:
# Preview.
df_ticker['daily_byWeekday'].head()

,date,open,high,low,close,adj close,volume,price_diff,year,week,weekday
0,1998-12-22,120.40625,121.218750,119.1875,120.68750,81.601967,5461100,NaN,1998,52,1
1,1998-12-23,121.18750,123.218750,120.8125,123.21875,83.313499,7791000,0.020974,1998,52,2
2,1998-12-24,123.15625,123.250000,122.5000,122.68750,82.954323,1507100,-0.004311,1998,52,3
3,1998-12-28,123.25000,123.312500,122.0000,122.37500,82.742989,4203600,-0.002548,1998,53,0
4,1998-12-29,122.71875,124.484375,122.1250,124.31250,84.053032,3935800,0.015833,1998,53,1


---

### Create pivot tables for price change.

Return to [Table of content](#Table-of-content.)

In [11]:
create_pivot(df_ticker, pivot_ticker, freq_keys, freq_cols, value='price_diff')

In [12]:
# Preview.
pivot_ticker['monthly'].head()

year,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
month,,,,,,,,,,,,,,,,,,,,,
1,0.038665,-0.047455,0.047662,-0.006393,-0.019801,0.024589,-0.017803,0.029443,0.020696,-0.055474,-0.074663,-0.031154,0.028682,0.053011,0.058646,-0.030011,-0.024274,-0.044131,0.023894,0.061758,0.086373
2,-0.032069,-0.015226,-0.095387,-0.017936,-0.013479,0.013571,0.020904,0.005725,-0.019618,-0.025842,-0.107449,0.031195,0.034737,0.043405,0.012759,0.045516,0.056205,-0.000826,0.039291,-0.036360,0.032416
3,0.038948,0.094134,-0.058572,0.030319,-0.001884,-0.016693,-0.022134,0.012478,0.007593,-0.013825,0.075611,0.056529,-0.004206,0.027660,0.033375,0.003865,-0.020080,0.061790,-0.003087,-0.031290,0.013636
4,0.040492,-0.032668,0.088362,-0.055457,0.088985,-0.015482,-0.014881,0.016659,0.048438,0.052849,0.107215,0.019652,0.033432,-0.002322,0.023745,0.011396,0.014341,0.009119,0.014320,0.009194,0.045437
5,-0.022866,-0.015723,-0.005605,-0.005934,0.054836,0.017124,0.032225,-0.030121,0.033920,0.015116,0.058453,-0.079454,-0.011215,-0.060056,0.023610,0.023206,0.012856,0.017012,0.014113,0.024309,-0.063771


In [13]:
# Preview.
pivot_ticker['weekly'].head()

year,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
week,,,,,,,,,,,,,,,,,,,,,
1,0.035986,-0.007660,-0.000640,0.045434,0.013947,NaN,-0.020104,0.031564,-0.024146,0.066544,NaN,0.028087,0.011054,0.031443,-0.005222,-0.014441,NaN,-0.058616,0.016463,0.021655,-0.001394
2,-0.026419,0.008362,0.021770,-0.022785,0.018719,0.010429,-0.001689,0.001869,0.019212,-0.008209,-0.041631,-0.008117,0.016989,0.008848,0.004783,0.006835,-0.005744,-0.021415,-0.000704,0.016458,0.026111
3,-0.014573,-0.017223,0.015270,-0.015573,-0.025790,0.016372,-0.012348,-0.021060,-0.002932,-0.057724,-0.045235,-0.038983,-0.007192,0.024138,0.008567,-0.002715,-0.012827,0.014429,-0.001365,0.008959,0.028882
4,0.041561,-0.059282,0.013874,0.003535,-0.047209,0.001751,0.005566,0.020402,-0.004832,0.007421,-0.022925,-0.016665,-0.005064,-0.000985,0.012944,-0.025866,0.016565,0.016796,0.009835,0.022003,-0.002552
5,-0.028152,0.049448,-0.007911,-0.007926,-0.003705,-0.008302,0.023844,-0.017660,0.018856,0.049158,-0.003369,-0.006798,0.026856,0.020634,0.006589,-0.003969,-0.026931,-0.029785,0.001616,-0.038837,0.016103


In [14]:
# Preview.
pivot_ticker['daily_byTrdrDay'].head()

year            1998      1999      2000      2001      2002      2003  \
month trdr_day                                                           
1     0          NaN -0.002281 -0.009787 -0.018104  0.010761  0.032189   
      1          NaN  0.011430 -0.039106  0.048035  0.011339  0.003075   
      2          NaN  0.024108  0.001789 -0.010764  0.006675  0.017625   
      3          NaN -0.004904 -0.016071 -0.032643 -0.007056 -0.002474   
      4          NaN  0.007393  0.058076  0.007740 -0.002312 -0.014450   

year                2004      2005      2006      2007      2008      2009  \
month trdr_day                                                               
1     0        -0.000449 -0.004715  0.017589 -0.001766 -0.008755  0.030142   
      1         0.010878 -0.012220  0.004736  0.002122 -0.000483 -0.001184   
      2         0.000978 -0.006901  0.000628 -0.007976 -0.024507  0.006677   
      3         0.003376  0.005084  0.008321  0.004625 -0.000849 -0.029956   
      4         0.003985 -0.001433  0.002569 -0.000850 -0.016148  0.004081   

year                2010      2011      2012      2013      2014      2015  \
month trdr_day                                                               
1     0         0.016960  0.010338  0.015936  0.025630 -0.009584 -0.000535   
      1         0.002647 -0.000551  0.001569 -0.002260 -0.000164 -0.018060   
      2         0.000704  0.005197  0.002662  0.004392 -0.002898 -0.009419   
      3         0.004222 -0.001958 -0.002577 -0.002733  0.006142  0.012461   
      4         0.003328 -0.001963  0.002428 -0.002877  0.000218  0.017745   

year                2016      2017      2018      2019      2020  
month trdr_day                                                    
1     0        -0.013980  0.007650  0.007157  0.001040  0.009352  
      1         0.001691  0.005949  0.006325 -0.023863 -0.007572  
      2        -0.012614 -0.000794  0.004215  0.033496       NaN  
      3        -0.023992  0.003578  0.006664  0.007885       NaN  
      4        -0.010976 -0.003301  0.001829  0.009395       NaN

In [15]:
# Preview.
pivot_ticker['daily_byWeekday'].head()

year          1998      1999      2000      2001      2002      2003  \
week weekday                                                           
1    0         NaN -0.002281 -0.009787 -0.014655  0.008354  0.013309   
     1         NaN  0.011430 -0.039106 -0.018104  0.001362  0.000180   
     2         NaN  0.024108  0.001789  0.048035  0.010761  0.000899   
     3         NaN -0.004904 -0.016071 -0.010764  0.011339  0.032189   
     4         NaN  0.007393  0.058076 -0.032643  0.006675  0.003075   

year              2004      2005      2006      2007      2008      2009  \
week weekday                                                               
1    0             NaN -0.004715       NaN -0.007400 -0.002868       NaN   
     1             NaN -0.012220  0.017589       NaN  0.023703       NaN   
     2             NaN -0.006901  0.004736 -0.001766  0.002760       NaN   
     3             NaN  0.005084  0.000628  0.002122 -0.000483       NaN   
     4       -0.000449 -0.001433  0.008321 -0.007976 -0.024507  0.030142   

year              2010      2011      2012      2013      2014      2015  \
week weekday                                                               
1    0        0.016960  0.010338  0.016996 -0.000163  0.001343       NaN   
     1        0.002647 -0.000551  0.015936  0.004733 -0.005366       NaN   
     2        0.000704  0.005197  0.001569  0.025630 -0.009923       NaN   
     3        0.004222 -0.001958  0.002662 -0.002260 -0.009584       NaN   
     4        0.003328 -0.001963 -0.002577  0.004392 -0.000164 -0.000535   

year              2016      2017      2018      2019      2020  
week weekday                                                    
1    0       -0.013980       NaN  0.008759 -0.005513       NaN  
     1        0.001691  0.007650  0.007157  0.002429       NaN  
     2       -0.012614  0.005949  0.006325  0.001040       NaN  
     3       -0.023992 -0.000794  0.004215 -0.023863  0.009352  
     4       -0.010976  0.003578  0.006664  0.033496 -0.007572

---

### Compute statistical summary on the pivot data, price change. 

Return to [Table of content](#Table-of-content.)

The function raises an error as there are specific rows that contain NaN for the entire columns. It's fine as this is expected. 

In [16]:
summarise_pivot(pivot_ticker, pivot_stats, freq_keys, start_yr_range, end_yr)

/Users/lioneltay/Applications/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


In [17]:
# Preview.
pivot_stats['monthly'].head()

,avg_diff,med_diff,tot_diff,max_diff,min_diff,std_diff,up_overall,pos_avg_diff,up_counts,neg_avg_diff,down_counts,up_prob,down_prob
month,,,,,,,,,,,,,
1,0.005822,0.020696,0.122258,0.086373,-0.074663,0.044685,1,0.043038,11,-0.035116,10,0.5238,0.4762
2,-0.001356,0.005725,-0.028471,0.056205,-0.107449,0.043272,0,0.030520,11,-0.036419,10,0.5238,0.4762
3,0.013532,0.007593,0.284168,0.094134,-0.058572,0.037603,1,0.037995,12,-0.019086,9,0.5714,0.4286
4,0.023944,0.016659,0.502826,0.107215,-0.055457,0.039804,1,0.038977,16,-0.024162,5,0.7619,0.2381
5,0.001525,0.014113,0.032035,0.058453,-0.079454,0.036934,1,0.027232,12,-0.032749,9,0.5714,0.4286


In [18]:
# Preview.
pivot_stats['weekly'].head()

,avg_diff,med_diff,tot_diff,max_diff,min_diff,std_diff,up_overall,pos_avg_diff,up_counts,neg_avg_diff,down_counts,up_prob,down_prob
week,,,,,,,,,,,,,
1,0.009442,0.012501,0.169955,0.066544,-0.058616,0.029439,1,0.030218,10,-0.016528,8,0.5556,0.4444
2,0.001127,0.004783,0.023670,0.026111,-0.041631,0.017772,1,0.013365,12,-0.015190,9,0.5714,0.4286
3,-0.007568,-0.007192,-0.158922,0.028882,-0.057724,0.022607,0,0.016660,7,-0.019681,14,0.3333,0.6667
4,-0.000625,0.003535,-0.013126,0.041561,-0.059282,0.023459,0,0.014354,12,-0.020598,9,0.5714,0.4286
5,0.001417,-0.003705,0.029760,0.049448,-0.038837,0.024189,1,0.023678,9,-0.015279,12,0.4286,0.5714


In [19]:
# Preview.
pivot_stats['daily_byTrdrDay'].head()

avg_diff  med_diff  tot_diff  max_diff  min_diff  std_diff  \
month trdr_day                                                               
1     0         0.005021  0.001040  0.105438  0.032189 -0.018104  0.014274   
      1         0.000567  0.001691  0.011907  0.048035 -0.039106  0.016407   
      2         0.001585  0.000978  0.033275  0.033496 -0.024507  0.012671   
      3        -0.002993 -0.000849 -0.062858  0.012461 -0.032643  0.012581   
      4         0.003070  0.001829  0.064477  0.058076 -0.016148  0.014787   

                up_overall  pos_avg_diff  up_counts  neg_avg_diff  \
month trdr_day                                                      
1     0                  1      0.015945         11     -0.006996   
      1                  1      0.009150         12     -0.010877   
      2                  1      0.008396         13     -0.009484   
      3                  0      0.006236         10     -0.011383   
      4                  1      0.009899         12     -0.006035   

                down_counts  up_prob  down_prob  
month trdr_day                                   
1     0                  10   0.5238     0.4762  
      1                   9   0.5714     0.4286  
      2                   8   0.6190     0.3810  
      3                  11   0.4762     0.5238  
      4                   9   0.5714     0.4286

In [20]:
# Preview.
pivot_stats['daily_byWeekday'].head()

avg_diff  med_diff  tot_diff  max_diff  min_diff  std_diff  \
week weekday                                                               
1    0        0.000919 -0.001222  0.014698  0.016996 -0.014655  0.010399   
     1        0.001245  0.002429  0.021160  0.023703 -0.039106  0.014609   
     2        0.006017  0.002274  0.108300  0.048035 -0.012614  0.014263   
     3       -0.001789 -0.000639 -0.032211  0.032189 -0.023992  0.012892   
     4        0.003901  0.003075  0.081916  0.058076 -0.032643  0.018990   

              up_overall  pos_avg_diff  up_counts  neg_avg_diff  down_counts  \
week weekday                                                                   
1    0                 1      0.010866          7     -0.006818            9   
     1                 1      0.008042         12     -0.015069            5   
     2                 1      0.009965         14     -0.007801            4   
     3                 0      0.007808          8     -0.009467           10   
     4                 1      0.015013         11     -0.008322           10   

              up_prob  down_prob  
week weekday                      
1    0         0.4375     0.5625  
     1         0.7059     0.2941  
     2         0.7778     0.2222  
     3         0.4444     0.5556  
     4         0.5238     0.4762

---

## Preprocess the volume data. 

### Configuration for volume data.

Return to [Table of content](#Table-of-content.)

This __configuration section focuses on volume__. I will also create __4 types__ of datasets, __monthly, weekly, daily (by trading day), and daily (by weekday)__. Different dataset will be used for different preprocessing procedure and purpose of analysis. 

This section will be saved in __config.py__ under __configuration__ folder to organise my task and file later.

In [21]:
# To store pivot tables.
pivot_volume = {
    freq_keys[0]: None, 
    freq_keys[1]: None, 
    freq_keys[2]: None, 
    freq_keys[3]: None
}

# To store statistical summary of pivot tables. 
pivot_volume_stats = {
    freq_keys[0]: None, 
    freq_keys[1]: None, 
    freq_keys[2]: None, 
    freq_keys[3]: None
}

---

### Functions for volume data preprocessing. 

Return to [Table of content](#Table-of-content.)

Below is the created functions __for preprocessing and creating a statistical summary__. Each function has its documentation. I __reduce the running time to O(N)__ as much as possible for python code except for pandas functions. The functions will be saved in __preprocessing.py__ under __autoprocess_ticker__ folder. 

In [22]:
def compute_avgVol(pivot_dict, pivot_dict_avg, freq_keys, start_yr_range, end_yr):
    '''
    Purpose: 
        Compute the average volume across columns and rows.
    
    Input  :
        pivot_dict      : Dictionary. To contain the pivot tables for volume.
        pivot_dict_avg  : Dictionary. To contain the average data from pivot_dict. 
        freq_keys       : List of dictionary keys. Must contain monthly/weekly/daily. 
        start_yr_range  : List. Range of starting year to summarise the data on. 
        end_yr          : Int. Ending year to summarise the data on. 
        
    Return :
        None.
        
    Note   :
        There are cells that contain NaN. A warning will raise if the entire cells 
        for calculating the mean or standard deviation are NaN. So far, the resulted 
        calculation contains no error. 
    '''

    for freq in freq_keys:
        repeated_start_yr = False
        
        for i, start_yr in enumerate(start_yr_range):
            if start_yr < start_yr_range[0]: 
                continue
            
            stats_key = f'{freq}_{yr_range[i]}'
            if start_yr == start_yr_range[0]:
                stats_key = freq 
                if repeated_start_yr:
                    continue
                repeated_start_yr = True
                
            # Filter the columns to specific year range. 
            pivot_dict_copy = pivot_dict[freq].loc[:,start_yr:end_yr].copy()

            # Compute the average volume across columns and rows. 
            pivot_dict_avg[f'{stats_key}_avgVolRow'] = pd.DataFrame(pivot_dict_copy.mean(axis=1), 
                                                                    columns=['avgVolRow'])
            pivot_dict_avg[f'{stats_key}_avgVolCol'] = pd.DataFrame(pivot_dict_copy.mean(axis=0), 
                                                                    columns=['avgVolCol'])



def summarise_pivot_vol(pivot_dict, pivot_dict_stats, freq_keys, start_yr_range, end_yr):
    '''
    Purpose: 
        Count the months that have volume above and below the average volume for each year. 
    
    Input  :
        pivot_dict      : Dictionary. To contain the pivot tables for volume.
        pivot_dict_stats: Dictionary. To contain the summarised data from pivot_dict. 
        freq_keys       : List of dictionary keys. Must contain monthly/weekly/daily. 
        start_yr_range  : List. Range of starting year to summarise the data on. 
        end_yr          : Int. Ending year to summarise the data on. 
        
    Return :
        None.
        
    Note   :
        There are cells that contain NaN. A warning will raise if the entire cells 
        for calculating the mean or standard deviation are NaN. So far, the resulted 
        calculation contains no error. 
    '''
        
    for freq in freq_keys:
        repeated_start_yr = False
        
        for i, start_yr in enumerate(start_yr_range):
            if start_yr < start_yr_range[0]: 
                continue
            
            stats_key = f'{freq}_{yr_range[i]}'
            if start_yr == start_yr_range[0]:
                stats_key = freq 
                if repeated_start_yr:
                    continue
                repeated_start_yr = True
                
            # List containing dataframes for 'pandas concat'. 
            ls_df = []

            for year in range(start_yr, end_yr + 1, 1):
                # Find rows of each year that are above the average volume. 
                df_filteredYear = pivot_dict[freq][[year]].copy()
                avgVol = pivot_dict_stats[f'{stats_key}_avgVolCol'].loc[year,:][0]
                abv_avgVol = df_filteredYear[year] > avgVol
                blw_avgVol = df_filteredYear[year] < avgVol

                # Indicate the rows that are above the average volume. 
                df_filteredYear[f'abv_avgVol_{year}'] = np.nan
                df_filteredYear.loc[abv_avgVol, f'abv_avgVol_{year}'] = 1 
                df_filteredYear.loc[blw_avgVol, f'abv_avgVol_{year}'] = 0 

                ls_df.append(df_filteredYear[[f'abv_avgVol_{year}']]) 

            # Perform pandas concat. 
            pivot_dict_stats[stats_key] = pd.concat(ls_df, axis=1)

            # Count the total monthly volume that are above or below the average volume. 
            totalCounts = pivot_dict_stats[stats_key].count(axis=1) 
            abvCounts = pivot_dict_stats[stats_key].sum(axis=1)
            pivot_dict_stats[stats_key]['abv_avgVolCounts'] = abvCounts
            pivot_dict_stats[stats_key]['blw_avgVolCounts'] = totalCounts - abvCounts
            pivot_dict_stats[stats_key]['abv_avgVolProb'] = abvCounts / totalCounts

---

### Create pivot tables for volume. 

Return to [Table of content](#Table-of-content.)

In [23]:
create_pivot(df_ticker, pivot_volume, freq_keys, freq_cols, value='volume')

In [24]:
pivot_volume.keys()

dict_keys(['monthly', 'weekly', 'daily_byTrdrDay', 'daily_byWeekday'])

In [25]:
# Preview.
pivot_volume['monthly'].head()

year,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
month,,,,,,,,,,,,,,,,,,,,,,
1,NaN,141419000.0,156770800.0,181296400.0,349380000.0,9.113199e+08,7.376740e+08,1.184212e+09,1.233911e+09,1.330330e+09,6.106834e+09,6.872039e+09,3.706842e+09,2.860315e+09,2.883821e+09,2.587140e+09,2.530651e+09,3.183506e+09,3.712951e+09,1.482408e+09,1.985507e+09,2.048692e+09
2,NaN,164624900.0,186938300.0,178607000.0,424492600.0,8.622483e+08,6.769769e+08,1.025608e+09,1.145244e+09,1.494549e+09,4.151029e+09,7.275092e+09,4.451912e+09,2.820074e+09,2.894582e+09,2.581459e+09,2.394185e+09,1.901638e+09,2.920731e+09,1.365137e+09,2.923722e+09,1.371716e+09
3,NaN,148191100.0,247594900.0,318187200.0,385578100.0,1.156870e+09,1.125004e+09,1.330549e+09,1.350777e+09,2.918304e+09,5.462122e+09,8.813960e+09,3.899688e+09,4.787459e+09,3.057516e+09,2.330972e+09,2.554084e+09,2.749923e+09,2.323306e+09,1.921474e+09,2.323562e+09,1.678081e+09
4,NaN,156755700.0,229246200.0,251839700.0,404461000.0,9.961142e+08,9.686209e+08,1.633318e+09,1.300328e+09,1.791290e+09,4.014653e+09,6.023007e+09,3.887090e+09,2.812718e+09,3.035644e+09,2.907035e+09,2.345902e+09,2.036685e+09,1.910636e+09,1.549614e+09,1.998466e+09,1.209205e+09
5,NaN,182566900.0,161024000.0,208040000.0,452626500.0,8.815099e+08,9.810839e+08,1.334647e+09,1.752747e+09,2.508178e+09,3.646997e+09,5.195736e+09,7.413879e+09,3.337607e+09,4.004156e+09,2.781596e+09,1.781094e+09,1.892520e+09,1.828531e+09,1.492547e+09,1.606397e+09,1.845593e+09


In [26]:
# Preview.
pivot_volume['weekly'].head()

year,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
week,,,,,,,,,,,,,,,,,,,,,,
1,NaN,36948400.0,42725700.0,59727300.0,140743500.0,111617300.0,NaN,294245300.0,235349700.0,455800050.0,7.176057e+08,NaN,6.040928e+08,688663800.0,670201550.0,344004400.0,404984400.0,NaN,9.085652e+08,320049800.0,449440300.0,243667300.0
2,NaN,45376100.0,32748700.0,35562900.0,68932300.0,190900900.0,169125700.0,289953500.0,223453000.0,329610500.0,1.465959e+09,1.444298e+09,7.595331e+08,587344200.0,625172800.0,566665900.0,483464500.0,809915800.0,1.147083e+09,320192700.0,337325100.0,471340300.0
3,NaN,27529300.0,24691300.0,32283100.0,83899400.0,176325100.0,186248000.0,246980300.0,323622400.0,220263000.0,1.535562e+09,2.001667e+09,1.046465e+09,593568300.0,560163500.0,591328800.0,533573200.0,940434000.0,8.458847e+08,311811500.0,461462000.0,457771900.0
4,NaN,31565200.0,45836400.0,39669000.0,49081700.0,203666400.0,134721800.0,300499700.0,344839100.0,310992100.0,1.477390e+09,1.598957e+09,1.296752e+09,841489200.0,751131900.0,510031400.0,491066100.0,545806600.0,8.114183e+08,374736400.0,515553600.0,357649000.0
5,NaN,44189200.0,38317400.0,39997600.0,108769500.0,263688600.0,209506200.0,253704500.0,339874900.0,347310200.0,1.272366e+09,1.599552e+09,1.427225e+09,715288600.0,744447200.0,596648600.0,821519600.0,765883700.0,8.440015e+08,384956600.0,593556800.0,434018300.0


In [27]:
# Preview.
pivot_volume['daily_byTrdrDay'].head()

year            1998       1999        2000        2001        2002  \
month trdr_day                                                        
1     0          NaN  9450400.0   8164300.0   8737500.0  18651900.0   
      1          NaN  8031000.0   8089800.0  19431600.0  15743000.0   
      2          NaN  7737700.0  12177900.0   9219000.0  20140700.0   
      3          NaN  5504900.0   6227200.0  12911400.0  13106500.0   
      4          NaN  6224400.0   8066500.0   6625300.0  12683700.0   

year                  2003        2004        2005        2006        2007  \
month trdr_day                                                               
1     0         44516300.0  38072300.0  55748000.0  73256700.0  94807600.0   
      1         32222600.0  27959800.0  69167600.0  51899600.0  69620600.0   
      2         40984500.0  20472800.0  65667300.0  47307500.0  76645300.0   
      3         38640400.0  30170400.0  47814700.0  62885900.0  71655000.0   
      4         38702200.0  36438400.0  55847700.0  43527400.0  75680100.0   

year                   2008         2009         2010         2011  \
month trdr_day                                                       
1     0         204935600.0  227566300.0  118944600.0  138725200.0   
      1         125133300.0  240349700.0  111579900.0  137409700.0   
      2         232330900.0  328260900.0  116074400.0  133975300.0   
      3         234991000.0  280899200.0  131091100.0  122519000.0   
      4         326365700.0  263834400.0  126402800.0  156034600.0   

year                   2012         2013         2014         2015  \
month trdr_day                                                       
1     0         193697900.0  192059000.0  119636900.0  121465900.0   
      1         127186500.0  144761800.0   81390600.0  169632600.0   
      2         173895000.0  116817700.0  108028200.0  209151400.0   
      3         148050000.0  110002500.0   86144200.0  125346700.0   
      4          99530200.0  121265100.0   96582300.0  147217800.0   

year                   2016        2017        2018         2019        2020  
month trdr_day                                                                
1     0         222353500.0  91366500.0  86655700.0  126925200.0  59151200.0  
      1         110845800.0  78744400.0  90070400.0  144140700.0  77709700.0  
      2         152112600.0  78379000.0  80636400.0  142628800.0         NaN  
      3         213436100.0  71559900.0  83524000.0  103139100.0         NaN  
      4         209817200.0  46939700.0  57319200.0  102512600.0         NaN

In [28]:
# Preview.
pivot_volume['daily_byWeekday'].head()

year          1998       1999        2000        2001        2002        2003  \
week weekday                                                                    
1    0         NaN  9450400.0   8164300.0  14619500.0  29968000.0  22483700.0   
     1         NaN  8031000.0   8089800.0   8737500.0  34036600.0  19559500.0   
     2         NaN  7737700.0  12177900.0  19431600.0  18651900.0  31501800.0   
     3         NaN  5504900.0   6227200.0   9219000.0  15743000.0  44516300.0   
     4         NaN  6224400.0   8066500.0  12911400.0  20140700.0  32222600.0   

year                2004        2005        2006         2007         2008  \
week weekday                                                                 
1    0               NaN  55748000.0         NaN  108126800.0  127795900.0   
     1               NaN  69167600.0  73256700.0          NaN  168256300.0   
     2               NaN  65667300.0  51899600.0   94807600.0  199461400.0   
     3               NaN  47814700.0  47307500.0   69620600.0  125133300.0   
     4        38072300.0  55847700.0  62885900.0   76645300.0  232330900.0   

year                 2009         2010         2011         2012         2013  \
week weekday                                                                    
1    0                NaN  118944600.0  138725200.0  243935200.0   56857000.0   
     1                NaN  111579900.0  137409700.0  193697900.0   86119900.0   
     2                NaN  116074400.0  133975300.0  127186500.0  192059000.0   
     3                NaN  131091100.0  122519000.0  173895000.0  144761800.0   
     4        227566300.0  126402800.0  156034600.0  148050000.0  116817700.0   

year                 2014         2015         2016        2017         2018  \
week weekday                                                                   
1    0         79643900.0          NaN  222353500.0         NaN  144299400.0   
     1         73540800.0          NaN  110845800.0  91366500.0   86655700.0   
     2        130333800.0          NaN  152112600.0  78744400.0   90070400.0   
     3        119636900.0          NaN  213436100.0  78379000.0   80636400.0   
     4         81390600.0  121465900.0  209817200.0  71559900.0   83524000.0   

year                 2019        2020  
week weekday                           
1    0         49729100.0         NaN  
     1         57077300.0         NaN  
     2        126925200.0         NaN  
     3        144140700.0  59151200.0  
     4        142628800.0  77709700.0

---

### Compute statistical summary on the pivot data, volume. 

Return to [Table of content](#Table-of-content.)

In [29]:
compute_avgVol(pivot_volume, pivot_volume_stats, freq_keys, start_yr_range, end_yr)

In [30]:
summarise_pivot_vol(pivot_volume, pivot_volume_stats, freq_keys, start_yr_range, end_yr)

In [31]:
# Preview.
pivot_volume_stats['monthly'][['abv_avgVolCounts','blw_avgVolCounts','abv_avgVolProb']].head()

year,abv_avgVolCounts,blw_avgVolCounts,abv_avgVolProb
month,,,
1,8.0,13.0,0.380952
2,9.0,12.0,0.428571
3,14.0,7.0,0.666667
4,10.0,11.0,0.476190
5,12.0,9.0,0.571429


In [32]:
# Preview.
pivot_volume_stats['weekly'][['abv_avgVolCounts','blw_avgVolCounts','abv_avgVolProb']].head()

year,abv_avgVolCounts,blw_avgVolCounts,abv_avgVolProb
week,,,
1,3.0,15.0,0.166667
2,6.0,15.0,0.285714
3,8.0,13.0,0.380952
4,12.0,9.0,0.571429
5,15.0,6.0,0.714286


In [33]:
# Preview.
pivot_volume_stats['daily_byTrdrDay'][['abv_avgVolCounts','blw_avgVolCounts','abv_avgVolProb']].head()

year            abv_avgVolCounts  blw_avgVolCounts  abv_avgVolProb
month trdr_day                                                    
1     0                     10.0              11.0        0.476190
      1                      9.0              12.0        0.428571
      2                     10.0              11.0        0.476190
      3                      6.0              15.0        0.285714
      4                      6.0              15.0        0.285714

In [34]:
# Preview.
pivot_volume_stats['daily_byWeekday'][['abv_avgVolCounts','blw_avgVolCounts','abv_avgVolProb']].head()

year          abv_avgVolCounts  blw_avgVolCounts  abv_avgVolProb
week weekday                                                    
1    0                     6.0              10.0        0.375000
     1                     8.0               9.0        0.470588
     2                     9.0               9.0        0.500000
     3                     7.0              11.0        0.388889
     4                     5.0              16.0        0.238095

---

## Preprocess the holidays, observances, and special periods data.

### Configuration for holidays, observances, and special periods data.

Return to [Table of content](#Table-of-content.)

This __configuration section focuses on holidays, observances, and special periods__. I categorise the items into different list for preprocessing. It helps to configure the items also if there is any changes. This section will be saved in __config.py__ under __configuration__ folder to organise my task and file later.

In [35]:
holidays_dict = {
    'newYear': [],
    'marLutKingJr': [], 
    'valentine': [], 
    'president': [], 
    'goodFriday': [],
    'memorial': [], 
    'independence': [], 
    'labour': [], 
    'event911': [], 
    'columbus': [], 
    'veteran': [], 
    'thanksgiving': [],
    'christmas': []
}

# Store keys for dictionary indexing. 
holidays_keys = [
    'newYear', 'marLutKingJr', 'valentine', 'president', 
    'goodFriday', 'memorial', 'independence', 'labour', 
    'event911', 'columbus', 'veteran', 'thanksgiving','christmas'
]
specialDays_keys = [
    'firstTrdrDoM', 'firstTrdrDoM_byMonth', 'superDay', 'superDay_byMonth',
    'santaRally', 'twwQ1', 'twwQ2', 'twwQ3', 'twwQ4', 
    'twwQ1_weekAft', 'twwQ2_weekAft', 'twwQ3_weekAft', 'twwQ4_weekAft'
]

# Holidays that falls on specific weekday. 
specWeekdayHolidays = ['marLutKingJr', 'president', 'memorial', 'labour', 'columbus', 'thanksgiving']

# Holidays that falls on Friday. 
specWeekdayHolidays_backward = ['goodFriday']

# Holidays that falls on specific date.
nonSpecHolidays = ['independence', 'christmas']

# Observances that falls on specific date.
nonSpecObservances = ['valentine', 'event911', 'veteran']


# To store pivot tables. 
pivot_holidays = {}
pivot_specialDays = {}
pivot_specialDays_weekly = {}

# To store statistical summary of pivot tables. 
pivot_holidays_stats = {}
pivot_specialDays_stats = {}
pivot_specialDays_weekly_stats ={}

### Function for tracing the holidays, observances, and special periods. 

Return to [Table of content](#Table-of-content.)

Below is the created functions. Each function has its documentation. I __reduce the running time to O(N)__ as much as possible for python code except for pandas functions. Some functions are build on top of the previous functions. The functions will be saved in __preprocessing.py__ under __autoprocess_ticker__ folder. 

The following functions is to __generate the specific dates for holidays, observances, special period (First Day of the Month, Santa Rally, TWW)__ within a specific range of years. It will then be used to trace the special period or the previous and following day after the holiday and observance dates. 

In [36]:
class trdrHolidays(AbstractHolidayCalendar):
    rules = [
        Holiday('NewYearDay', month=1, day=1, observance=None),
        USMartinLutherKingJr,
        Holiday('Valentine', month=2, day=14, observance=None),
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday('USIndependenceDay', month=7, day=4, observance=None),
        USLaborDay,
        Holiday('USEvent911', month=9, day=11, observance=None),
        USColumbusDay,
        Holiday('USVeterans', month=11, day=11, observance=None),
        USThanksgivingDay,
        Holiday('Christmas', month=12, day=25, observance=None)
    ]

    
    
def get_trdrHoliday_date(year):
    '''
    Purpose: 
        Identify the date of the holidays or special days. 
    
    Input  :
        year: Int. The specific year for identifying the 
              holiday and special day date. 
        
    Return :
        List of holiday and special day dates for specific year.
    '''
    
    inst = trdrHolidays()
    return inst.holidays(datetime(year, 1, 1), datetime(year, 12, 31)) 



def compile_trdrHoliday_dates(start_yr, end_yr, holidays_keys):
    '''
    Purpose: 
        Compile all the holiday and special day dates into a dataframe 
        for further analysis later. 
    
    Input  :
        start_yr: Int. Starting year.
        end_yr  : Int. Ending year. 
        
    Return :
        Dateframe containing all the holiday and special day dates.
    '''
    
    generated_holidays = []
    
    # Compile the dates into a list of list.
    for year in range(start_yr, end_yr + 1, 1):
        generated_holidays.append(get_trdrHoliday_date(year))
        
    return pd.DataFrame(generated_holidays, columns=holidays_keys) 



def get_tww_dates(start_yr, end_yr):
    '''
    Purpose: 
        Compile all the Triple Witching Week (TWW) dates into a 
        dataframe for further analysis later. 
    
    Input  :
        start_yr: Int. Starting year.
        end_yr  : Int. Ending year. 
        
    Return :
        Dateframe containing all the Triple Witching Week (TWW) dates.
        
    Note   :
        TWW only appears on Friday of the third week of each quarter. 
    '''

    tww_dict = {'twwQ1': [], 'twwQ2': [], 'twwQ3': [], 'twwQ4': []}

    for year in range(start_yr, end_yr + 1,1):
        for i, month in enumerate([3,6,9,12]):
            for day in range(15,22,1):
                date = datetime(year,month,day)
                # In python weekday = 4 means Friday. 
                if date.weekday() == 4: 
                    tww_dict[f'twwQ{i + 1}'].append(date)
                    break 
                    
    return pd.DataFrame(tww_dict)



def get_santaRally_period(df_ticker_data, start_yr, end_yr):
    '''
    Purpose: 
        Compile all the Santa Rally dates into a 
        list for further analysis later. 
    
    Input  :
        df_ticker_data: Dataframe. Must be 'daily_byTrdrDay'. 
        start_yr      : Int. Starting year.
        end_yr        : Int. Ending year. 
        
    Return :
        Tuple containing all the Santa Rally dates, day counts, and specific year 
        of each period. 
        
    Note   :
        Santa Rally occurs at the last five trading days of the current year
        and the first two trading days of the next year. 
    '''
    
    ls_santaRally = []
    ls_santaRally_period = []
    ls_santaRally_dayCounts = []
    ls_santaRally_specYear = []
    ls_daily_dates = df_ticker_data['date'].dt.date.tolist()
    
    # Find the last trading date of each year. 
    for year in range(start_yr - 1, end_yr + 1,1):
        for day in range(31,24,-1):
            date = datetime(year,12,day)
            if date.weekday() != 5 and date.weekday() != 6:
                ls_santaRally.append(date)
                break
                
    # Find rest of the trading days starting from the last trading day of each year. 
    for date in ls_santaRally:
        year = date.date().year
        idx = ls_daily_dates.index(date.date()) - 4
        
        for i in range(0,7,1):
            idx_date = idx + i
            ls_santaRally_dayCounts.append(i)
            ls_santaRally_specYear.append(year) 
            ls_santaRally_period.append(datetime.combine(ls_daily_dates[idx_date], datetime.min.time()))
            
    return ls_santaRally_period, ls_santaRally_dayCounts, ls_santaRally_specYear



def get_superDay_period(df_ticker_data, date_range=5):
    '''
    Purpose: 
        Compile all the Super Day dates into a 
        list for further analysis later. 
    
    Input  :
        df_ticker_data: Dataframe. Must be 'daily_byTrdrDay'. 
        date_range    : Int. Total number of dates to indicate. 
                        If 6, then 3 trading days before the Super Day plus 
                        2 trading days after the Super Day. 
        
    Return :
        Tuple containing all the Super Day dates, day counts, and specific year 
        of each period. 
        
    Note   :
        Super Day period may change depending on the market. 
    '''
    
    ls_superDay_period = []
    ls_superDay_dayCounts = []
    ls_superDay_specMonth = []
    ls_superDay_specYear = []
    ls_daily_dates = df_ticker_data['date'].dt.date.tolist()
    
    # Find the first trading date of each month. 
    # Remove the first (start_yr - 1) and last date (end_yr + 1). 
    ls_daily_firstTrdrDate = df_ticker_data.loc[df_ticker_data['trdr_day'] == 0, 'date'].dt.date.tolist()[1:-1]
                
    # Find rest of the trading days starting from the last trading day of each year. 
    for date in ls_daily_firstTrdrDate:
        month = date.month
        year = date.year
        idx = ls_daily_dates.index(date) - 3
        
        for i in range(0,date_range,1):
            idx_date = idx + i
            ls_superDay_dayCounts.append(i) 
            ls_superDay_specMonth.append(month) 
            ls_superDay_specYear.append(year) 
            ls_superDay_period.append(datetime.combine(ls_daily_dates[idx_date], datetime.min.time()))
            
    return ls_superDay_period, ls_superDay_dayCounts, ls_superDay_specMonth, ls_superDay_specYear

---

The following functions is to __to trace the special period or the previous and following day after the holiday and observance dates__. Some holidays fall on specific weekday while others fall on specific dates, and observances are not holiday. Hence, different functions are created to serve different purpose. 

In [37]:
def trace_specialDays(df_ticker_data, tup_superDay, tup_santaRally):
    '''
    Purpose:
        Add columns to the dataframe with the following: 
            1. Indicate the 'First Trading of the Month'.
            2. Indicate the Super Day period, day counts, and specific year.
            3. Indicate the Santa Rally period, day counts, and specific year.
    
    Input  :
        df_ticker_data  : Dataframe. Must be 'daily_byTrdrDay'. 
        tup_superDay    : Tuple of Super Day dates, day counts, and specific year.
        tup_santaRally  : Tuple of Santa Rally dates, day counts, and specific year.
        
    Return :
        None. 
    '''
    
    df_ticker_data['firstTrdrDoM'] = 0
    # Remove the first (start_yr - 1) and last date (end_yr + 1). 
    ls_daily_firstTrdrDate = df_ticker_data.loc[df_ticker_data['trdr_day'] == 0,'date'].dt.date.tolist()[1:-1]
    df_ticker_data.loc[df_ticker_data['date'].isin(ls_daily_firstTrdrDate), 'firstTrdrDoM'] = 1 
    
    # Assign tuples to 'period' and 'dayCounts' variable. 
    ls_superDay_period, ls_superDay_dayCounts, ls_superDay_specMonth, ls_superDay_specYear = tup_superDay
    ls_santaRally_period, ls_santaRally_dayCounts, ls_santaRally_specYear = tup_santaRally
    
    # Indicate rows that fall within the special day period. 
    df_ticker_data['superDay'] = 0  
    df_ticker_data.loc[df_ticker_data['date'].isin(ls_superDay_period), 'superDay'] = 1

    df_ticker_data['santaRally'] = 0 
    df_ticker_data.loc[df_ticker_data['date'].isin(ls_santaRally_period), 'santaRally'] = 1
    
    # Add the day counts and specific month or year for each period. 
    period_bool = df_ticker_data['superDay'] == 1 
    df_ticker_data.loc[period_bool, 'superDay_dayCounts'] = ls_superDay_dayCounts 
    df_ticker_data.loc[period_bool, 'superDay_specMonth'] = ls_superDay_specMonth
    df_ticker_data.loc[period_bool, 'superDay_specYear'] = ls_superDay_specYear
    
    period_bool = df_ticker_data['santaRally'] == 1 
    df_ticker_data.loc[period_bool, 'santaRally_dayCounts'] = ls_santaRally_dayCounts 
    df_ticker_data.loc[period_bool, 'santaRally_specYear'] = ls_santaRally_specYear
    
    
    
def trace_tww_trdrDays(df_ticker_data, df_tww):
    '''
    Purpose: 
        Trace all the Triple Witching Week (TWW) trading days. 
    
    Input  :
        df_ticker_data: Dataframe. Must be 'daily_byTrdrDay' or 'weekly'.
        df_tww        : Dateframe containing all the TWW dates. 
        
    Return :
        None. 
        
    Note   :
        There is a Christmas holiday the following week after TWW 
        on quarter 4.  
    '''

    tww_trdrDays_dict = {} 
    quarters = df_tww.columns
    
    for quarter in quarters:
        # Create new columns. 
        df_ticker_data[quarter] = 0
        df_ticker_data[f'{quarter}_weekAft'] = 0
        
        # Compile the years and weeks into an array.
        tww_years = df_tww[quarter].dt.year.values
        tww_weeks = df_tww[quarter].dt.week.values
        
        for year, week in zip(tww_years, tww_weeks):
            # Perform filter on specific year and week. 
            year_bool = df_ticker_data['year'] == year
            week_bool = df_ticker_data['week'] == week
            weekAft_bool = df_ticker_data['week'] == week + 1
            
            # Indicate the tradings days that fall within TWW and the week after. 
            df_ticker_data.loc[year_bool & week_bool, quarter] = 1 
            df_ticker_data.loc[year_bool & weekAft_bool, f'{quarter}_weekAft'] = 1 
        
        # Compute the day counts for each TWW period of each quarter. 
        insert_dayCountsCol(df_ticker_data, quarter)
        insert_dayCountsCol(df_ticker_data, f'{quarter}_weekAft')


    
def insert_dayCountsCol(df_ticker_data, period_col):
    '''
    Purpose:
        Create a new column for day counts of specific holiday, 
        observance, or special day period. 
    
    Input  :
        df_ticker_data: Dataframe. Contains ticker data. Must be 'daily_byTrdrDay'. 
        period_col    : Str. Specific column of holiday, observance, or special day period. 
        
    Return :
        None. 
    '''
    
    period_bool = df_ticker_data[period_col] == 1
    df_ticker_data.loc[period_bool, f'{period_col}_dayCounts'] = df_ticker_data[period_bool]\
                                                                 .set_index(keys='date')\
                                                                 .groupby(by=pd.Grouper(freq='M'))\
                                                                 .cumcount().values



def insert_holidayCol(df_ticker_data, holidays_dict, holiday_col):
    '''
    Purpose:
        Create a new column from a holiday list. 
    
    Input  :
        df_ticker_data: Dataframe. Contains ticker data. Must be 'daily_byTrdrDay'. 
        holidays_dict : Dictionary. Contains list of dates within for
                        for each holiday. 
        holiday_col   : Str. Name of the holiday to create new column.
        
    Return :
        None. 
    '''
    
    df_ticker_data[holiday_col] = 0 
    df_ticker_data.loc[df_ticker_data['date'].isin(holidays_dict[holiday_col]),holiday_col] = 1
    
    # Add the day counts for each period of each year. 
    period_bool = df_ticker_data[holiday_col] == 1 
    df_ticker_data.loc[period_bool, f'{holiday_col}_dayCounts'] = holidays_dict[f'{holiday_col}_dayCounts']
    
    # Add the specific year for each period. 
    df_ticker_data.loc[period_bool, f'{holiday_col}_specYear'] = holidays_dict[f'{holiday_col}_specYear'] 
    

    
def trace_newYear(df_ticker_data, holidays_dict, date_range=6): 
    '''
    Purpose:
        Trace all the New Year period. 
    
    Input  :
        df_ticker_data: Dataframe. Contains ticker data. Must be 'daily_byTrdrDay'. 
        holidays_dict : Dictionary. Contains list of dates within for
                        for each holiday. 
        date_range    : Int. Total number of dates to indicate. 
                        If 6, then 3 trading days before the holiday plus 
                        3 trading days after the holiday. 
        
    Return :
        None. 
    '''
    
    holiday_col = 'newYear'
    holidays_dict[f'{holiday_col}_dayCounts'] = [] 
    holidays_dict[f'{holiday_col}_specYear'] = [] 
    
    # Remove the first (start_yr - 1) and last date (end_yr + 1). 
    month_bool = df_ticker_data['month'] == 1
    firstTrdrDate_bool = df_ticker_data['trdr_day'] == 0
    
    ls_daily_firstTrdrDate = df_ticker_data.loc[month_bool & firstTrdrDate_bool,'date'].dt.date.tolist()[:-1]
    ls_daily_dates = df_ticker_data['date'].dt.date.tolist()
    
    for date in ls_daily_firstTrdrDate: 
        year = date.year
        idx = ls_daily_dates.index(date) - 3 
        
        for i in range(0,date_range,1): 
            idx_date = idx + i 
            holidays_dict[f'{holiday_col}_dayCounts'].append(i) 
            holidays_dict[f'{holiday_col}_specYear'].append(year)
            holidays_dict[holiday_col].append(datetime.combine(ls_daily_dates[idx_date], 
                                                               datetime.min.time())) 
            
    insert_holidayCol(df_ticker_data, holidays_dict, holiday_col)


            
def trace_specWeekdayHoliday(df_ticker_data, df_holidays, holidays_dict, holiday_col, 
                             day_forward=1, idx_backtrace=3, date_range=6): 
    '''
    Purpose:
        Trace all the holidays period which happens on a specific weekday. 
    
    Input  :
        df_ticker_data: Dataframe. Contains ticker data. Must be 'daily_byTrdrDay'. 
        df_holidays   : Dataframe. Contains all the holiday dates. 
        holidays_dict : Dictionary. Contains list of dates within for
                        for each holiday. 
        holiday_col   : Str. Name of the holiday to create new column.
        day_forward   : Int. Move forward N number of days (not trading days).
        idx_backtrace : Int. Move backward N number of trading days.
        date_range    : Int. Total number of dates to indicate. 
                        If 6, then 3 trading days before the holiday plus 
                        3 trading days after the holiday. 
        
    Return :
        None. 
    ''' 
        
    holidays_dict[f'{holiday_col}_dayCounts'] = []
    holidays_dict[f'{holiday_col}_specYear'] = [] 
    ls_daily_dates = df_ticker_data['date'].dt.date.tolist() 
    
    for date in df_holidays[holiday_col].tolist(): 
        year = date.date().year
        date = date + timedelta(days=day_forward) 
        idx = ls_daily_dates.index(date.date()) - idx_backtrace 
        
        for i in range(0,date_range,1): 
            idx_date = idx + i 
            holidays_dict[f'{holiday_col}_dayCounts'].append(i)
            holidays_dict[f'{holiday_col}_specYear'].append(year)
            holidays_dict[holiday_col].append(datetime.combine(ls_daily_dates[idx_date], 
                                                               datetime.min.time())) 
    
    insert_holidayCol(df_ticker_data, holidays_dict, holiday_col)
    

    
def trace_nonSpecHoliday(df_ticker_data, df_holidays, holidays_dict, holiday_col, date_range=6): 
    '''
    Purpose:
        Trace all the holidays period which happens on a non-specific weekday. 
    
    Input  :
        df_ticker_data: Dataframe. Contains ticker data. Must be 'daily_byTrdrDay'. 
        df_holidays   : Dataframe. Contains all the holiday dates. 
        holidays_dict : Dictionary. Contains list of dates within for
                        for each holiday. 
        holiday_col   : Str. Name of the holiday to create new column.
        date_range    : Int. Total number of dates to indicate. 
                        If 6, then 3 trading days before the holiday plus 
                        3 trading days after the holiday. 
        
    Return :
        None. 
        
    Note   :
        If holiday falls on weekend, one day before or after the holiday 
        will be a holiday, depending on which is nearest. 
    ''' 
    
    holidays_dict[f'{holiday_col}_dayCounts'] = []
    holidays_dict[f'{holiday_col}_specYear'] = [] 
    ls_daily_dates = df_ticker_data['date'].dt.date.tolist() 
    
    for date in df_holidays[holiday_col].tolist(): 
        year = date.date().year
        
        if date.weekday() == 4: 
            date = date - timedelta(days=1) 
            idx = ls_daily_dates.index(date.date()) - 2
        elif date.weekday() == 5:
            date = date - timedelta(days=2)
            idx = ls_daily_dates.index(date.date()) - 2
        elif date.weekday() == 6:
            date = date + timedelta(days=2)
            idx = ls_daily_dates.index(date.date()) - 3
        else:
            date = date + timedelta(days=1)
            idx = ls_daily_dates.index(date.date()) - 3
            
        for i in range(0,date_range,1): 
            idx_date = idx + i 
            holidays_dict[f'{holiday_col}_dayCounts'].append(i)
            holidays_dict[f'{holiday_col}_specYear'].append(year)
            holidays_dict[holiday_col].append(datetime.combine(ls_daily_dates[idx_date], 
                                                               datetime.min.time())) 
        
    insert_holidayCol(df_ticker_data, holidays_dict, holiday_col)
        

        
def trace_nonSpecObservance(df_ticker_data, df_holidays, holidays_dict, holiday_col, date_range=7):
    '''
    Purpose:
        Trace all the observances period which happens on a non-specific weekday. 
    
    Input  :
        df_ticker_data: Dataframe. Contains ticker data. Must be 'daily_byTrdrDay'. 
        df_holidays   : Dataframe. Contains all the holiday dates. 
        holidays_dict : Dictionary. Contains list of dates within for
                        for each holiday. 
        holiday_col   : Str. Name of the holiday to create new column.
        date_range    : Int. Total number of dates to indicate. 
                        If 7, then 3 trading days before the observance plus
                        the day of the observance itself (if it's not on weekend)
                        plus 3 trading days after the observance. 
        
    Return : 
        None. 
    
    Note   :
        If the observances date falls on weekend, move one/two day forward until 
        it falls on weekday. If the day after is a holiday then move one 
        more day forward. Please be aware that the observance's effect might lost its 
        significance if the day is too far from the observance date. It will also affect 
        the overall statistical computation making its effect less even if there's really 
        a true effect. 
    ''' 
    
    holidays_dict[f'{holiday_col}_dayCounts'] = []
    holidays_dict[f'{holiday_col}_specYear'] = [] 
    ls_daily_dates = df_ticker_data['date'].dt.date.tolist() 
    
    for date in df_holidays[holiday_col].tolist():
        year = date.date().year
        date_range = 7
        
        # Try running this to see if the date falls on weekend, holiday, or trading day. 
        # If error occurs due to weekend or holiday, run the 'except' section. 
        while True:
            try:
                idx = ls_daily_dates.index(date.date()) - 3
                for i in range(0,date_range,1):
                    idx_date = idx + i 
                    if i >= 3 and date_range == 6:
                        i += 1
                    holidays_dict[f'{holiday_col}_dayCounts'].append(i)
                    holidays_dict[f'{holiday_col}_specYear'].append(year)
                    holidays_dict[holiday_col].append(datetime.combine(ls_daily_dates[idx_date], 
                                                                       datetime.min.time())) 
                break
            except:
                date = date + timedelta(days=1)
                date_range = 6
                
    insert_holidayCol(df_ticker_data, holidays_dict, holiday_col)

---

Below is the created functions __for preprocessing and creating a statistical summary__. Another function is for __compiling the TWW dates__.

In [38]:
def create_pivot_uniqueDays(df_ticker_data, pivot_dict, pivot_dict_keys, 
                            start_yr, end_yr):
    '''
    Purpose: 
        Create pivot tables for holidays, observances, and special day.
    
    Input  :
        df_ticker_data  : Dataframe. Must be 'daily_byTrdrDay'.
        pivot_dict      : Dictionary. To contain pivot tables.
        pivot_dict_keys : List of dictionary contains holidays, observances, 
                          or special day keys. 
        start_yr        : Int. Starting year to compile the data on. 
        end_yr          : Int. Ending year to compile the data on. 

    Return :
        None.
        
    Note   :
        1. Any special day like 'firstTrdrDoM' and 'superDay' which occur monthly 
           should include month as part of the groupby. Otherwise it will average 
           across all the different months. However, this depends on the analysis goal. 
        2. 'firstTrdrDoM' does not contain '_dayCounts'. 
        3. 'tww' does not contain '_specYear'.
        4. 'superDay' requires '_specMonth' instead of 'month' if groupby includes month. 
    '''
        
    for periodCol in pivot_dict_keys:       
        specYear = f'{periodCol}_specYear'
        idxCol = f'{periodCol}_dayCounts'
        
        if periodCol == 'firstTrdrDoM': 
            specYear = 'year'
            idxCol = periodCol
        elif 'tww' in periodCol:
            specYear = 'year'
        
        if 'byMonth' in periodCol:
            idxCol = periodCol[:-8]
            
            if idxCol == 'superDay': 
                specYear = f'{idxCol}_specYear' 
                idxCol = [f'{idxCol}_specMonth', f'{idxCol}_dayCounts']
            elif idxCol == 'firstTrdrDoM':
                specYear = 'year'
                idxCol = ['month', idxCol] 
            
        pivot_dict[periodCol] = df_ticker_data.pivot_table(values='price_diff', 
                                                           index=idxCol, columns=specYear, 
                                                           aggfunc='mean').loc[:,start_yr:end_yr]
        
        
        
def concat_pivot_tww(pivot_dict, pivot_dict_keys):
    '''
    Purpose: 
        Concat pivot tables for TWW data.
    
    Input  :
        pivot_dict     : Dictionary. Must be contain TWW dataframes.
        pivot_dict_keys: List. TWW dictionary indexing keys. 

    Return :
        None.
    '''
    
    for key in pivot_dict_keys:
        tww_key = key 
        tww_weekAft_key = f'{key}_weekAft'
        
        # Concat the 'TWW' and 'TWW week after' dataframes. 
        ls_df = [pivot_dict[tww_key], pivot_dict[tww_weekAft_key]]
        df_concat = pd.concat(ls_df, keys=('tww', 'tww_weekAft'), names=('twwPeriod', 'dayCounts'))
        
        # Update the TWW data. 
        pivot_dict[tww_key] = df_concat

---

### Get the holiday, observance, and special period dates.

Return to [Table of content](#Table-of-content.)

In [39]:
# Create a copy to avoid overwriting the original dataframes. 
df_ticker_weekly = df_ticker['weekly'].copy()
df_ticker_trdrDay = df_ticker['daily_byTrdrDay'].copy()

In [40]:
# List holidays, observances, and special day dates. 
df_holidays = compile_trdrHoliday_dates(start_yr, end_yr, holidays_keys)
df_tww = get_tww_dates(start_yr, end_yr)
tup_superDay = get_superDay_period(df_ticker_trdrDay) 
tup_santaRally = get_santaRally_period(df_ticker_trdrDay, start_yr, end_yr)

In [41]:
# Preview.
df_holidays.head()

,newYear,marLutKingJr,valentine,president,goodFriday,memorial,independence,labour,event911,columbus,veteran,thanksgiving,christmas
0,1999-01-01,1999-01-18,1999-02-14,1999-02-15,1999-04-02,1999-05-31,1999-07-04,1999-09-06,1999-09-11,1999-10-11,1999-11-11,1999-11-25,1999-12-25
1,2000-01-01,2000-01-17,2000-02-14,2000-02-21,2000-04-21,2000-05-29,2000-07-04,2000-09-04,2000-09-11,2000-10-09,2000-11-11,2000-11-23,2000-12-25
2,2001-01-01,2001-01-15,2001-02-14,2001-02-19,2001-04-13,2001-05-28,2001-07-04,2001-09-03,2001-09-11,2001-10-08,2001-11-11,2001-11-22,2001-12-25
3,2002-01-01,2002-01-21,2002-02-14,2002-02-18,2002-03-29,2002-05-27,2002-07-04,2002-09-02,2002-09-11,2002-10-14,2002-11-11,2002-11-28,2002-12-25
4,2003-01-01,2003-01-20,2003-02-14,2003-02-17,2003-04-18,2003-05-26,2003-07-04,2003-09-01,2003-09-11,2003-10-13,2003-11-11,2003-11-27,2003-12-25


In [42]:
# Preview.
df_tww.head()

,twwQ1,twwQ2,twwQ3,twwQ4
0,1999-03-19,1999-06-18,1999-09-17,1999-12-17
1,2000-03-17,2000-06-16,2000-09-15,2000-12-15
2,2001-03-16,2001-06-15,2001-09-21,2001-12-21
3,2002-03-15,2002-06-21,2002-09-20,2002-12-20
4,2003-03-21,2003-06-20,2003-09-19,2003-12-19


---

### Trace the FirstTrdrDoM and Santa Rally dates. 

Return to [Table of content](#Table-of-content.)

In [43]:
# Ticker 'daily_trdrDay' data. 
trace_specialDays(df_ticker_trdrDay, tup_superDay, tup_santaRally) 
trace_tww_trdrDays(df_ticker_trdrDay, df_tww)

# Ticker weekly data. 
trace_tww_trdrDays(df_ticker_weekly, df_tww)

---

### Trace the holiday and observance dates. 

Return to [Table of content](#Table-of-content.)

In [44]:
trace_newYear(df_ticker_trdrDay, holidays_dict) 

trace_specWeekdayHoliday(df_ticker_trdrDay, df_holidays, holidays_dict, 'marLutKingJr')
trace_specWeekdayHoliday(df_ticker_trdrDay, df_holidays, holidays_dict, 'president')
trace_specWeekdayHoliday(df_ticker_trdrDay, df_holidays, holidays_dict, 'memorial') 
trace_specWeekdayHoliday(df_ticker_trdrDay, df_holidays, holidays_dict, 'goodFriday', 
                         day_forward=-1, idx_backtrace=2)
trace_specWeekdayHoliday(df_ticker_trdrDay, df_holidays, holidays_dict, 'labour')
trace_specWeekdayHoliday(df_ticker_trdrDay, df_holidays, holidays_dict, 'columbus') 
trace_specWeekdayHoliday(df_ticker_trdrDay, df_holidays, holidays_dict, 'thanksgiving')

trace_nonSpecHoliday(df_ticker_trdrDay, df_holidays, holidays_dict, 'independence') 
trace_nonSpecHoliday(df_ticker_trdrDay, df_holidays, holidays_dict, 'christmas') 

trace_nonSpecObservance(df_ticker_trdrDay, df_holidays, holidays_dict, 'valentine')
trace_nonSpecObservance(df_ticker_trdrDay, df_holidays, holidays_dict, 'event911')
trace_nonSpecObservance(df_ticker_trdrDay, df_holidays, holidays_dict, 'veteran')

---

### Create pivot tables for holidays, observances, and special periods. 

Return to [Table of content](#Table-of-content.)

In [45]:
create_pivot_uniqueDays(df_ticker_trdrDay, pivot_holidays, holidays_keys, start_yr, end_yr)
create_pivot_uniqueDays(df_ticker_trdrDay, pivot_specialDays, specialDays_keys, start_yr, end_yr)
create_pivot_uniqueDays(df_ticker_weekly, pivot_specialDays_weekly, specialDays_keys[5:], start_yr, end_yr)

In [46]:
concat_pivot_tww(pivot_specialDays, specialDays_keys[5:9])
concat_pivot_tww(pivot_specialDays_weekly, specialDays_keys[5:9])

In [47]:
# Preview.
pivot_holidays['newYear']

newYear_specYear,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0
newYear_dayCounts,,,,,,,,,,,,,,,,,,,,,
0.0,0.015833,0.004276,0.007320,0.006068,-0.022486,0.013309,0.001485,0.002232,0.006569,-0.012571,-0.002868,-0.001419,0.000715,-0.013123,-0.001341,-0.000054,0.001343,0.010672,-0.008265,0.000486,0.007677
1.0,-0.008044,-0.001170,0.003047,-0.000517,0.008354,0.000180,-0.001895,-0.004453,-0.002105,-0.002506,0.023703,-0.000355,-0.001588,0.010334,-0.010808,-0.000163,-0.005366,-0.007088,-0.000223,0.002058,-0.001290
2.0,0.000000,0.001598,-0.018929,-0.014655,0.001362,0.000899,-0.002146,-0.005432,-0.004149,-0.007400,0.014275,-0.009598,0.000239,-0.004916,0.016996,0.004733,-0.009923,-0.010003,-0.003655,-0.003771,0.008759
3.0,-0.002281,-0.009787,-0.018104,0.010761,0.032189,-0.000449,-0.004715,0.017589,-0.001766,-0.008755,0.030142,0.016960,0.010338,0.015936,0.025630,-0.009584,-0.000535,-0.013980,0.007650,0.007157,0.001040
4.0,0.011430,-0.039106,0.048035,0.011339,0.003075,0.010878,-0.012220,0.004736,0.002122,-0.000483,-0.001184,0.002647,-0.000551,0.001569,-0.002260,-0.000164,-0.018060,0.001691,0.005949,0.006325,-0.023863
5.0,0.024108,0.001789,-0.010764,0.006675,0.017625,0.000978,-0.006901,0.000628,-0.007976,-0.024507,0.006677,0.000704,0.005197,0.002662,0.004392,-0.002898,-0.009419,-0.012614,-0.000794,0.004215,0.033496


In [48]:
# Preview.
pivot_holidays['goodFriday']

goodFriday_specYear,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0
goodFriday_dayCounts,,,,,,,,,,,,,,,,,,,,,
0.0,-0.005242,0.026421,0.018244,0.005810,0.009331,-0.003210,-0.010161,-0.008478,0.010763,0.041543,-0.023325,0.000682,0.005744,-0.004089,0.008003,0.006888,-0.008739,-0.000538,-0.001190,-0.017012,0.000655
1.0,-0.016048,-0.009302,0.000686,0.002626,-0.017042,-0.002350,0.000855,0.001866,0.001113,-0.024770,0.010777,-0.003407,0.013632,-0.009911,0.000000,0.010478,-0.003536,-0.006599,-0.004382,-0.002955,-0.002447
2.0,0.007547,0.004804,0.018162,-0.000436,0.014844,-0.002268,0.001197,-0.001319,0.002711,0.018523,0.039743,0.006838,0.005109,-0.000500,0.003073,0.001397,0.003598,-0.000443,-0.006495,0.012778,0.001969
3.0,0.023677,-0.010865,-0.010517,0.000437,0.001005,0.003935,0.001451,-0.000389,0.001386,0.019988,0.000234,0.008150,-0.001047,-0.011231,-0.003957,0.003487,0.006733,0.000591,0.008860,-0.021585,0.000862
4.0,-0.002360,0.041520,0.014116,-0.005499,0.018851,-0.014022,-0.006649,0.015856,0.001177,0.000965,-0.017244,0.002358,0.008605,-0.016785,0.004934,0.004545,-0.002647,0.009250,-0.002984,0.012817,0.008992
5.0,0.008043,-0.011285,0.039745,-0.007021,0.009197,0.001590,0.014160,0.001913,-0.004080,-0.012236,0.010670,-0.005712,0.006529,0.008094,-0.010139,-0.002342,0.003377,0.004388,-0.001839,0.010699,-0.002219


In [49]:
# Preview.
pivot_holidays['veteran']

veteran_specYear,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0
veteran_dayCounts,,,,,,,,,,,,,,,,,,,,,
0.0,0.000906,-0.022174,-0.001335,0.012956,0.005290,-0.001450,0.000000,0.002164,-0.027356,-0.055411,0.002620,-0.001874,0.012831,-0.022664,0.005106,0.004003,-0.000523,0.004504,0.001701,0.021409,0.000228
1.0,-0.009398,-0.003779,0.003118,-0.024506,-0.007424,-0.001964,0.001309,-0.005256,-0.005071,0.033018,0.022776,-0.007184,-0.036910,-0.012024,-0.012643,0.000935,-0.009332,0.010602,-0.003628,-0.001815,0.003517
2.0,0.007429,-0.024325,0.001066,-0.012230,-0.004071,0.000770,0.007762,0.000435,-0.013726,-0.013104,0.000183,0.004029,0.009419,0.000870,0.013491,0.003147,0.002307,0.002495,-0.000310,-0.009768,0.002466
3.0,0.005673,NaN,NaN,-0.015505,-0.000286,0.007609,0.003405,NaN,NaN,-0.030876,0.005110,-0.003767,0.018822,NaN,0.000169,0.000981,-0.003932,-0.002305,NaN,NaN,-0.001910
4.0,0.009025,-0.007777,-0.006121,0.007931,0.011222,0.007891,-0.000565,0.002459,-0.009922,-0.044002,-0.010168,-0.011838,-0.009474,0.000796,-0.002030,-0.001078,-0.013960,0.000785,0.000930,-0.018685,0.002108
5.0,0.002348,0.026280,0.022494,0.001012,0.000282,0.002457,-0.003638,0.007505,0.030480,0.062340,0.005411,-0.001414,0.004942,-0.003472,0.008024,0.001128,-0.011228,0.007803,-0.002323,-0.001871,0.000324
6.0,0.008366,0.003144,0.000960,0.018866,-0.008462,-0.007159,0.002028,0.002865,-0.002769,-0.049907,0.014504,-0.015580,-0.015863,-0.013499,0.004990,0.000245,0.015207,-0.001878,-0.005005,-0.006837,0.001456


In [50]:
# Preview.
pivot_specialDays['twwQ1']

year                       1999      2000      2001      2002      2003  \
twwPeriod   dayCounts                                                     
tww         0.0        0.014251 -0.010928 -0.042801  0.002137  0.031499   
            1.0       -0.003810 -0.014205  0.016430 -0.000597  0.005877   
            2.0       -0.004303  0.023330 -0.019747 -0.009644  0.007675   
            3.0        0.016087  0.046714  0.000255 -0.001378  0.002160   
            4.0       -0.016998  0.006609 -0.020057  0.009528  0.021345   
tww_weekAft 0.0        0.001927 -0.005104  0.020346  0.000171 -0.033233   
            1.0       -0.028860  0.020522 -0.026843  0.006686  0.009575   
            2.0        0.005696  0.006075 -0.016988 -0.018817 -0.005028   
            3.0        0.020438  0.017073 -0.010155  0.000434  0.000804   
            4.0       -0.007240  0.005937  0.030238 -0.002168 -0.005049   

year                       2004      2005      2006      2007      2008  \
twwPeriod   dayCounts                                                     
tww         0.0       -0.012258  0.006230  0.001867  0.001492 -0.010107   
            1.0        0.005306 -0.008255  0.010479 -0.019434  0.041543   
            2.0        0.011181 -0.008490  0.004456  0.007450 -0.024770   
            3.0        0.000265  0.002015  0.002065  0.001364  0.018523   
            4.0       -0.014333 -0.002969  0.000835 -0.002800       NaN   
tww_weekAft 0.0       -0.012696 -0.003712 -0.001607  0.012055  0.019988   
            1.0       -0.001733 -0.010161 -0.006288  0.005492  0.000965   
            2.0        0.000823  0.000855  0.006096  0.016457 -0.012236   
            3.0        0.013236  0.001197 -0.002071 -0.000768 -0.003154   
            4.0        0.000270       NaN  0.000769  0.001467 -0.009564   

year                       2009      2010      2011      2012      2013  \
twwPeriod   dayCounts                                                     
tww         0.0       -0.003022  0.000260 -0.006038  0.000073  0.003795   
            1.0        0.030582  0.007966 -0.011457  0.018026 -0.002243   
            2.0        0.022384  0.005927 -0.018513 -0.001071  0.001413   
            3.0       -0.012386 -0.000512  0.013235  0.005790  0.005324   
            4.0       -0.021294 -0.005062  0.003637  0.001385 -0.001320   
tww_weekAft 0.0        0.071829  0.005346  0.015498  0.003920 -0.005519   
            1.0       -0.019703  0.007033 -0.003469 -0.002911 -0.002323   
            2.0        0.010546 -0.004855  0.002862 -0.001637  0.006985   
            3.0        0.020380 -0.001626  0.009563 -0.007204 -0.008543   
            4.0       -0.018048 -0.000600  0.003056  0.003233  0.008033   

year                       2014      2015      2016      2017      2018  \
twwPeriod   dayCounts                                                     
tww         0.0        0.009044  0.013360 -0.001282  0.000505 -0.001255   
            1.0        0.007138 -0.002972 -0.001630 -0.003827 -0.006463   
            2.0       -0.005329  0.012021  0.005787  0.008653 -0.005132   
            3.0        0.005840 -0.004561  0.006344 -0.001967 -0.001090   
            4.0       -0.003878  0.008827  0.003930 -0.001756  0.001084   
tww_weekAft 0.0       -0.004136 -0.001948  0.001419 -0.001097 -0.013531   
            1.0        0.004746 -0.005619 -0.000538 -0.012840  0.001701   
            2.0       -0.007192 -0.014654 -0.006599  0.002353 -0.001919   
            3.0       -0.002108 -0.002381 -0.000443 -0.001067 -0.024997   
            4.0        0.004930  0.002289       NaN -0.000727 -0.021315   

year                       2019  
twwPeriod   dayCounts            
tww         0.0        0.014501  
            1.0        0.003771  
            2.0        0.006619  
            3.0       -0.000640  
            4.0        0.004941  
tww_weekAft 0.0        0.003626  
            1.0        0.000248  
            2.0       -0.003010  
            3.0        0.011295  
            4.0       

---

### Compute statistical summary on the pivot data; holidays, observances, and special periods. 

Return to [Table of content](#Table-of-content.)

In [51]:
summarise_pivot(pivot_holidays, pivot_holidays_stats, holidays_keys, start_yr_range, end_yr)
summarise_pivot(pivot_specialDays, pivot_specialDays_stats, specialDays_keys, start_yr_range, end_yr)
summarise_pivot(pivot_specialDays_weekly, pivot_specialDays_weekly_stats, specialDays_keys[5:9], 
                start_yr_range, end_yr)

In [52]:
# Preview.
pivot_holidays_stats['newYear']

,avg_diff,med_diff,tot_diff,max_diff,min_diff,std_diff,up_overall,pos_avg_diff,up_counts,neg_avg_diff,down_counts,up_prob,down_prob
newYear_dayCounts,,,,,,,,,,,,,
0.0,0.000755,0.001343,0.015858,0.015833,-0.022486,0.009128,1,0.005999,13,-0.007766,8,0.6190,0.3810
1.0,0.000005,-0.001170,0.000104,0.023703,-0.010808,0.007255,1,0.007946,6,-0.003171,15,0.2857,0.7143
2.0,-0.002177,-0.003655,-0.045717,0.016996,-0.018929,0.008715,0,0.006108,8,-0.007881,12,0.4000,0.6000
3.0,0.005021,0.001040,0.105438,0.032189,-0.018104,0.014274,1,0.015945,11,-0.006996,10,0.5238,0.4762
4.0,0.000567,0.001691,0.011907,0.048035,-0.039106,0.016407,1,0.009150,12,-0.010877,9,0.5714,0.4286
5.0,0.001585,0.000978,0.033275,0.033496,-0.024507,0.012671,1,0.008396,13,-0.009484,8,0.6190,0.3810


In [53]:
# Preview.
pivot_holidays_stats['goodFriday']

,avg_diff,med_diff,tot_diff,max_diff,min_diff,std_diff,up_overall,pos_avg_diff,up_counts,neg_avg_diff,down_counts,up_prob,down_prob
goodFriday_dayCounts,,,,,,,,,,,,,
0.0,0.002481,0.000655,0.052099,0.041543,-0.023325,0.014406,1,0.012189,11,-0.008198,10,0.5238,0.4762
1.0,-0.002891,-0.002447,-0.060716,0.013632,-0.024770,0.009226,0,0.005254,8,-0.008562,12,0.4000,0.6000
2.0,0.006230,0.003073,0.130829,0.039743,-0.006495,0.010150,1,0.009486,15,-0.001910,6,0.7143,0.2857
3.0,0.001010,0.000862,0.021205,0.023677,-0.021585,0.010001,1,0.005771,14,-0.008513,7,0.6667,0.3333
4.0,0.003609,0.002358,0.075796,0.041520,-0.017244,0.013390,1,0.011076,13,-0.008524,8,0.6190,0.3810
5.0,0.002930,0.001913,0.061532,0.039745,-0.012236,0.011434,1,0.009867,12,-0.006319,9,0.5714,0.4286


In [54]:
# Preview.
pivot_holidays_stats['veteran']

,avg_diff,med_diff,tot_diff,max_diff,min_diff,std_diff,up_overall,pos_avg_diff,up_counts,neg_avg_diff,down_counts,up_prob,down_prob
veteran_dayCounts,,,,,,,,,,,,,
0.0,-0.002813,0.000906,-0.059068,0.021409,-0.055411,0.016677,0,0.006143,12,-0.016598,8,0.6000,0.4000
1.0,-0.003127,-0.003779,-0.065657,0.033018,-0.036910,0.014453,0,0.010754,7,-0.010067,14,0.3333,0.6667
2.0,-0.001032,0.000870,-0.021667,0.013491,-0.024325,0.009011,0,0.003991,14,-0.011076,7,0.6667,0.3333
3.0,-0.001201,-0.000058,-0.016810,0.018822,-0.030876,0.011419,0,0.005967,7,-0.008368,7,0.5000,0.5000
4.0,-0.004403,-0.001078,-0.092472,0.011222,-0.044002,0.012152,0,0.004794,9,-0.011302,12,0.4286,0.5714
5.0,0.007566,0.002348,0.158883,0.062340,-0.011228,0.016158,1,0.012189,15,-0.003991,6,0.7143,0.2857
6.0,-0.002587,0.000245,-0.054327,0.018866,-0.049907,0.014405,0,0.006603,11,-0.012696,10,0.5238,0.4762


In [55]:
# Preview.
pivot_specialDays_stats['twwQ1']

avg_diff  med_diff  tot_diff  max_diff  min_diff  \
twwPeriod   dayCounts                                                     
tww         0.0        0.000539  0.000505  0.011322  0.031499 -0.042801   
            1.0        0.003439 -0.000597  0.072224  0.041543 -0.019434   
            2.0        0.000948  0.004456  0.019899  0.023330 -0.024770   
            3.0        0.004926  0.002015  0.103447  0.046714 -0.012386   
            4.0       -0.001417 -0.000243 -0.028347  0.021345 -0.021294   
tww_weekAft 0.0        0.003502  0.000171  0.073543  0.071829 -0.033233   
            1.0       -0.003063 -0.001733 -0.064319  0.020522 -0.028860   
            2.0       -0.001628 -0.001637 -0.034187  0.016457 -0.018817   
            3.0        0.001424 -0.000768  0.029903  0.020438 -0.024997   
            4.0       -0.001249  0.000270 -0.023735  0.030238 -0.021315   

                       std_diff  up_overall  pos_avg_diff  up_counts  \
twwPeriod   dayCounts                                                  
tww         0.0        0.014100           1      0.007616         13   
            1.0        0.014449           1      0.014712         10   
            2.0        0.012660           1      0.009742         12   
            3.0        0.011796           1      0.008999         14   
            4.0        0.010505           0      0.006212         10   
tww_weekAft 0.0        0.019631           1      0.014193         11   
            1.0        0.011783           0      0.006330          9   
            2.0        0.009120           0      0.005875         10   
            3.0        0.010863           1      0.010491          9   
            4.0        0.011455           0      0.006022         10   

                       neg_avg_diff  down_counts  up_prob  down_prob  
twwPeriod   dayCounts                                                 
tww         0.0           -0.010961            8   0.6190     0.3810  
            1.0           -0.006809           11   0.4762     0.5238  
            2.0           -0.010778            9   0.5714     0.4286  
            3.0           -0.003219            7   0.6667     0.3333  
            4.0           -0.009047           10   0.5000     0.5000  
tww_weekAft 0.0           -0.008258           10   0.5238     0.4762  
            1.0           -0.010107           12   0.4286     0.5714  
            2.0           -0.008449           11   0.4762     0.5238  
            3.0           -0.005376           12   0.4286     0.5714  
            4.0           -0.009329            9   0.5263     0.4737

---

## Save the data into Excel file on multiple sheets. 

### Configuration for writing to Excel file. 

Return to [Table of content](#Table-of-content.)

In [56]:
sheetNames = ['mth', 'wk', 'trdr', 'wkd']

# Excel row and column index. 
startcol = 1
startrow = 2
distance = 2

### Function for saving multiple data into Excel for verification.

Return to [Table of content](#Table-of-content.)

I have not included a section to verify the computation and preprocessed data as it requires time to write the code. Hence, I opt for a manual approach by verifying them in Excel on multiple sheets. 

In [57]:
def singleSheetMultiWrite(excelWriter, pivot_dict, pivot_stats, sheetName, keys, 
                          startcol, startrow, distance):
    import regex as re
    
    startrow1st = startrow
    
    for i, key in enumerate(keys):
        pivot_dict[key].to_excel(excelWriter, sheet_name=sheetName, 
                                 startcol=startcol, startrow=startrow1st)
        startcol2nd = startcol + len(pivot_dict[key].columns) + distance
        
        re_compile = re.compile(f'(?:{key})(?:_R.*Yr)?(?!_)')
        stats_keys = list(filter(re_compile.match, pivot_stats.keys()))
        
        for stats_key in stats_keys:
            pivot_stats[stats_key].to_excel(excelWriter, sheet_name=sheetName, 
                                            startcol=startcol2nd, startrow=startrow1st)
            startcol2nd = startcol2nd + len(pivot_stats[stats_key].columns) + distance
        
        startrow1st = startrow1st + len(pivot_dict[key].index) + distance 


        
def multiSheetWrite(excelWriter, pivot_dict, pivot_stats, sheetName, key, 
                    startcol, startrow, distance):    
    import regex as re
    
    pivot_dict[key].to_excel(excelWriter, sheet_name=sheetName, 
                             startcol=startcol, startrow=startrow)
    startcol2nd = startcol + len(pivot_dict[key].columns) + distance

    re_compile = re.compile(f'(?:{key})(?:_R.*Yr)?(?!_)')
    stats_keys = list(filter(re_compile.match, pivot_stats.keys()))
    
    for stats_key in stats_keys:
        pivot_stats[stats_key].to_excel(excelWriter, sheet_name=sheetName, 
                                        startcol=startcol2nd, startrow=startrow) 
        startcol2nd = startcol2nd + len(pivot_stats[stats_key].columns) + distance

---

### Save to Excel. 

Return to [Table of content](#Table-of-content.)

In [58]:
with pd.ExcelWriter(f'{ETF_folder}/{ticker}/{ticker}_seasonal_stats.xlsx') as writer:
    
    # Save pivot ticker data and volume. Multiple sheets. 
    for i, freq in enumerate(freq_keys):         
        multiSheetWrite(writer, pivot_ticker, pivot_stats, f'{sheetNames[i]}_pv', freq, 
                        startcol, startrow, distance)
        multiSheetWrite(writer, pivot_volume, pivot_volume_stats, f'{sheetNames[i]}_vol', 
                        freq, startcol, startrow, distance)        
    
    # Save pivot tables holidays/observances data. Single sheet.
    singleSheetMultiWrite(writer, pivot_holidays, pivot_holidays_stats, 'holi', holidays_keys, 
                          startcol, startrow, distance)
        
    # Save pivot tables TWW Weekly data. Single sheet.
    singleSheetMultiWrite(writer, pivot_specialDays_weekly, pivot_specialDays_weekly_stats, 
                          'twwWk', specialDays_keys[5:9], startcol, startrow, distance)
        
    # Save pivot tables TWW Trdr data. Single sheet.
    singleSheetMultiWrite(writer, pivot_specialDays, pivot_specialDays_stats, 
                          'twwTrdr', specialDays_keys[5:9], startcol, startrow, distance)
    
    # Save pivot tables Special Day data. Single sheet. 
    singleSheetMultiWrite(writer, pivot_specialDays, pivot_specialDays_stats, 
                          'specialD', specialDays_keys[:5], startcol, startrow, distance)

---

## Simple file verification. 

Return to [Table of content](#Table-of-content.)

To check whether the data is stored properly. 

In [59]:
import pickle

with open(f'{project_path}/{ETF_folder}/{ticker}/storage/pivot_uniqueDays.pickle','rb') as read_file:
    df_testView = pickle.load(read_file)
    
df_testView[0][0].keys()

dict_keys(['newYear', 'marLutKingJr', 'valentine', 'president', 'goodFriday', 'memorial', 'independence', 'labour', 'event911', 'columbus', 'veteran', 'thanksgiving', 'christmas'])

## Notes for future reference. 

Return to [Table of content](#Table-of-content.)

---

Latest update: __2020/05/01__

__List of things to take note:__

- The `create_pivot_uniqueDays` function is not well designed. 
- The function `compute_avgVol` does not compute the average by each month of each year separately for `daily_byTrdrDay` and `daily_byWeekday` as I don't think it contributes to the analysis. However, if changes are made, the rest of the functions will have to be updated also. This is due to the indexing level issue. 
- Consider computing the sum of the price change for `firstTrdr_byMonth` and `superDay_byMonth` for each year other than averaging them across 12 months. 
- Consider computing the sum of the `nonSuperDay` and `nonSuperDay_byMonth` to compare with the total gain from `superDay`. 
- The data hasn't been verified for `20Yr`, `15Yr`, `10Yr`, and `5Yr` range. 
- The total file size for the ticker data increases 6 times due to the 'storage' folder containing all the __.pickle__ files for Luigi pipeline. Consider an alternative format to compress the data. 